In [1]:
# CAGE spectrum check
# 13 Oct 2022

# code sources -- CAGE/analysis/nplus.ipynb
#              -- CAGE/processing/energy_raw.ipynb

In [1]:
import os, sys, h5py, json, time
import pandas as pd
import numpy as np
import h5py
from scipy.optimize import curve_fit
from scipy.ndimage.filters import gaussian_filter

# use this to get interactive plots at NERSC.  
# requires ipympl & jupyter-matplotlib extension for jupyterlab
# user may need to $pip install ipympl --update
# %matplotlib widget
# Successfully installed ipympl-0.9.2

import matplotlib.pyplot as plt

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from tqdm import tqdm
    tqdm.pandas() # suppress annoying FutureWarning

/tmp/ipykernel_21510/4115107063.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [2]:
# from pygama import DataGroup, lh5

# imports for pygama tag v1.1.0, oct 2022.
from pygama.flow import DataGroup
import pygama.lgdo.lh5_store as lh5
import pygama.math.histogram as pgh
    
# import pygama.analysis.histograms as pgh
# import pygama.analysis.peak_fitting as pgf

# from pygama.dsp.dsp_optimize import *
# from pygama.dsp.WaveformBrowser import WaveformBrowser as wfb
# from pygama.io.hit_to_evt import cluster_events

In [3]:
dg = DataGroup('../processing/cage.json', load=True)    

# que = 'cycle >=2563 and cycle <= 2565' # oppi in cage recently
# que = 'run >= 414 and run <= 417' 
que = 'run == 425'
# que = 'run == 430'

df_cycles = dg.fileDB.query(que)

# # df_runs.columns
# # ['unique_key', 'YYYY', 'mm', 'dd', 'cycle', 'daq_dir', 'daq_file', 'run',
# #        'runtype', 'detector', 'skip', 'dsp_id', 'raw_file', 'raw_path',
# #        'dsp_file', 'dsp_path', 'hit_file', 'hit_path', 'startTime',
# #        'threshold', 'daq_gb', 'stopTime', 'runtime']

view = ['run','cycle','daq_file','startTime','threshold','daq_gb']
display(df_cycles[view])

Warning, DAQ directory not found: $CAGE_DAQ
Warning, LH5 directory not found: $CAGE_LH5
Warning, LH5 user directory not found: $CAGE_LH5_USER
Warning, run selection file not found: $CAGE_SW/processing/metadata/run_selection.json
environ({'LEGENDDATADIR': '/global/project/projectdirs/m2676/data', 'PYTHONUNBUFFERED': '1', 'PYTHONIOENCODING': 'utf-8', 'SHIFTER_IMAGEREQUEST': 'legendexp/legend-base:latest', 'SHIFTER_IMAGE': '00d58d1bb58f22ae23b4d4c65f29bff8209f2c86a92a197a8b6ae300cf4dcff4', 'PATH': '/opt/root/bin:/opt/geant4/bin:/opt/clhep/bin:/opt/hdf5/bin:/opt/nodejs/bin:/opt/anaconda3/bin:/opt/anaconda3/condabin:/opt/julia/bin:/opt/julia-1.9/bin:/opt/julia-1.8/bin:/opt/julia-1.6/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/udiImage/bin', 'NVARCH': 'x86_64', 'NVIDIA_REQUIRE_CUDA': 'cuda>=11.7 brand=tesla,driver>=450,driver<451 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470

FileNotFoundError: File $CAGE_SW/processing/fileDB.h5 does not exist

In [4]:
# check the first raw file and show the available LH5 tables and column names
raw_files = dg.lh5_dir + df_cycles['raw_path'] + '/' + df_cycles['raw_file']

# h5py has a funny way of iterating through groups, you have to pass 'visititems' a function
def print_attrs(name, obj):
    for key, val in obj.attrs.items():
        if 'table' in val: # debug, only show tables.
            print(name)
            print("    %s: %s" % (key, val))

print('Raw file 0:')
with h5py.File(raw_files.iloc[0]) as hf:
    print(raw_files.iloc[0])
    hf.visititems(print_attrs)

NameError: name 'dg' is not defined

In [5]:
# load enough data to make an energy spectrum in both channels, and save some other interesting variables too

def load_events(row, tb_name, data_cols, tb_type):
    """
    I should add this function to DataGroup in pygama, it's like GATDataSet:GetGatifiedChain
    """
    # print(row)
    lh5_file = dg.lh5_dir + row[f'{tb_type}_path'] + '/' + row[f'{tb_type}_file']
    lh5_data = pd.DataFrame(lh5.load_nda(lh5_file, data_cols, tb_name))
    lh5_data['cycle'] = row['cycle']
    return lh5_data

# data_cols = ['channel','timestamp','energy','trapEmax','trapEftp','bl','bl_sig','bl_slope','bl_intercept',
#              'wf_max','wf_argmax','tp_0','dcr','A_10','triE','hf_max','lf_max','tp_max']

data_cols = ['energy','energy_first', 'timestamp']

ncycles = 10
tb_type = 'raw'
etype = 'energy'

tmp = df_cycles[:ncycles].progress_apply(load_events, axis=1, args=('ORSIS3302DecoderForEnergy/raw', data_cols, tb_type,))

# tmp = df_cycles.progress_apply(load_events, axis=1, args=('ORSIS3302DecoderForEnergy/raw', data_cols, tb_type,))

df_hits = pd.concat([df for df in tmp])
print('In-memory size:', round(sys.getsizeof(df_hits) / 1024 / 1024, 2), "MB")

# df_hits['energy'].max()
df_hits



NameError: name 'df_cycles' is not defined

In [6]:
# %matplotlib widget


# histogram energy data for this estimator -- TODO: and normalize by runtime

# uncalibrated plot
xlo, xhi, xpb = 0, 4e6, 10000   # good for energy
data = df_hits[etype]
hist, bins, var = pgh.get_hist(data, range=(xlo, xhi), dx=xpb)

# # # rough calibrated plot.
# elo, ehi, epb = 1, 250, 0.5 # 1--250 keV range
elo, ehi, epb = 1, 2800, 0.5 

# # run 414, one point calibration - 1460 keV = 1.954e6 in `energy`.
# # cal = 1460 / 1.954e6
cal = 1460 / 1.928e6
# cal = 1460 / 1.830e6 # oppi in cage sep 2022
data_cal = df_hits[etype] * cal
hist, bins, var = pgh.get_hist(data_cal, range=(elo, ehi), dx=epb)

bins = bins[1:] # trim zero bin, not needed with ds='steps'

# hist_rt = np.divide(hist, runtime_min * 60)
# print(f'\nRaw E: {etype}, {len(data)} cts, runtime: {runtime_min:.2f} min')

plt.semilogy(bins, hist, ds='steps', c='b', lw=1, label=etype)
# plt.plot(bins, hist, ds='steps', c='b', lw=1, label=etype)

plt.xlabel(etype, ha='right', x=1)
# plt.ylabel(f'cts/sec, {xpb}/bin', ha='right', y=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()


NameError: name 'df_hits' is not defined